In [16]:
#!/usr/bin/env python3
import subprocess

def main():
    MODEL = "o3-mini"
    # 0부터 33까지 반복
    failed = [10,28]
    # for target in range(34):
    for target in failed:

        outdir = f"./results/success_desc"
        cmd = [
            "python", "generate_descriptions.py",
            "--target", str(target),
            "--samples", "1",
            "--intergrated",
            "--outdir", outdir,
            "--model", MODEL,
            "--num_generations", "2",
            "--max_tokens", "40000",
            "--batch_size", "1",
            "--num_descriptions", "75",
            "--rng_offset", "777",
        ]
        print(f"\n▶ Running target={target}")
        print("  ", " ".join(cmd))
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ Error for target {target}:")
            print(result.stdout)
            print(result.stderr)
        else:
            print(f"✅ Completed target {target}")

In [ ]:
main()

In [ ]:
#!/usr/bin/env python3
import os
import glob
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed

# 1. 파라미터 설정
TEST = "float-line"                 # --test 에 들어갈 값
MODEL = "o3-mini"                  # --prompt_model 에 들어갈 값
TARGET = "0"                       # 출력 디렉터리 명에 포함될 값

# 2. 처리할 파일 목록 준비 (.jsonl 파일을 모두 가져오는 예시)
#    필요에 따라 glob 패턴을 바꾸거나, 직접 리스트를 정의하세요.
filenames = glob.glob("./results/success_desc/*.jsonl")

# 3. 출력 디렉터리 생성
output_dir = f"./results/success_code/"
os.makedirs(output_dir, exist_ok=True)

def process_file(filename):
    """
    단일 파일에 대해 subprocess로 generate_code.py를 실행하고,
    (filename, returncode, stdout, stderr)를 리턴합니다.
    """
    cmd = [
        "python", "generate_code.py",
        "--test", TEST,
        "--outdir", output_dir,
        "--ignore_cache_samples",
        "--prompt_model", MODEL,
        "--max_tokens", "40000",
        "-n", "2",
        "-s", "4",
        "--nohtml",
        "--jsonl", filename,
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return filename, result.returncode, result.stdout, result.stderr

def main():
    max_workers = 300  # 동시에 띄울 프로세스 수
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Future 객체와 filename 매핑
        futures = {executor.submit(process_file, fn): fn for fn in filenames}
        
        # 완료되는 순서대로 결과 처리
        for future in as_completed(futures):
            fn = futures[future]
            try:
                filename, code, out, err = future.result()
                if code == 0:
                    print(f"✅ SUCCESS: {filename}")
                else:
                    print(f"❌ FAIL:    {filename} (exit {code})")
                    print(f"--- stdout ---\n{out}")
                    print(f"--- stderr ---\n{err}")
            except Exception as e:
                print(f"❌ EXCEPTION: {fn} -> {e}")

if __name__ == "__main__":
    main()

In [30]:
input_pattern = f"/home/hyunseok/BARC/results/success_code/*.jsonl"
filenames = glob.glob(input_pattern)

In [1]:
import os
import glob
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed

# 1. 파라미터 설정
TEST = "float-line"      # --test 에 들어갈 값
MODEL = "o3-mini"        # --prompt_model 에 들어갈 값
TARGET = "0"             # 출력 디렉터리 명에 포함될 값

# 2. 입력 파일 목록 준비
input_pattern = f"/home/hyunseok/BARC/results/success_code/*.jsonl"
filenames = glob.glob(input_pattern)
if not filenames:
    print(f"[WARN] 입력 파일을 찾을 수 없습니다: {input_pattern}")
    exit(1)

# 3. 출력 및 실패 로그 디렉터리 생성
output_dir  = f"/home/hyunseok/BARC/results/success_problem"
failure_dir = f"/home/hyunseok/BARC/results/failure_problem"
os.makedirs(output_dir,  exist_ok=True)
os.makedirs(failure_dir, exist_ok=True)

def log_error(jsonl_name, error_message, gif_id, result_path):
    from hyeonseok_utils.generate_metadata_desc import generate_metadata_csv_of_step_descriptions
    METADATA_CSV_PATH='/home/hyunseok/BARC/results/metadata/step_problem_metadata.csv'
    with open(result_path, "w") as f:
        import uuid
        from datetime import datetime, timezone
        step_id = str(uuid.uuid4())
        try:
            sample_records = [{
                "id": step_id,
                "step_name": "problem",
                "prev_step_id": "GIF",
                "gif_id": str(gif_id),
                "gen_model": "o3-mini",
                "result_code":0,
                "result_path": result_path,
                "error_message": str(error_message),
                "createAt": datetime.now(timezone.utc),   # ← 현재 UTC 시각
                # "token_usage": {"prompt": 142, "completion": 23},
            }]
            generate_metadata_csv_of_step_descriptions( sample_records, output_csv=METADATA_CSV_PATH )
        except Exception as e:
            print(f"an error occurred: {jsonl_name}, {e}")

def process_file(fn: str):
    """
    단일 .jsonl 파일에 대해 generate_problems.py를 실행합니다.
    """
    cmd = [
        "python", "generate_problems.py",
        "--jsonl", fn,
        "--outdir", output_dir,
        "--total_timeout", "300"
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return fn, result.returncode, result.stdout, result.stderr

def main():
    folder = "/home/hyunseok/BARC/results/success_problem"
    csv_file = "/home/hyunseok/BARC/data/records.csv"
    csv_file='/home/hyunseok/BARC/results/metadata/step_problem_metadata.csv'
    base_dir='/home/hyunseok/BARC/results/success_problem'
    # 1) 빈 .jsonl 파일 찾기
    # empty_list = find_empty_jsonl(folder)
    # empty_file_path = [ os.path.join(base_dir, ffpath) for ffpath in empty_list]
    # print("빈 파일 목록:", empty_file_path)

    # 동시 실행 워커 수 (파일 개수나 머신 사양에 맞게 조절)
    max_workers = min(12, len(filenames))
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_file, fn): fn for fn in filenames}
        for future in as_completed(futures):
            fn = futures[future]
            try:
                filename, code, out, err = future.result()
                if code == 0:
                    print(f"✅ SUCCESS: {filename}")
                else:
                    print(f"❌ FAIL:    {filename} (exit {code})")
                    # 실패 로그 파일로 저장
                    base = os.path.splitext(os.path.basename(filename))[0]
                    log_path = os.path.join(failure_dir, f"{base}.log")
                    with open(log_path, "w", encoding="utf-8") as logf:
                        logf.write(f"COMMAND: python generate_problems.py --jsonl {filename} --outdir {output_dir} --total_timeout 300\n")
                        logf.write(f"EXIT CODE: {code}\n\n")
                        logf.write("--- STDOUT ---\n")
                        logf.write(out + "\n")
                        logf.write("--- STDERR ---\n")
                        logf.write(err + "\n")
                    print(f"    👉 로그 저장: {log_path}")
            except Exception as e:
                print(f"❌ EXCEPTION: {fn} -> {e}")

if __name__ == "__main__":
    main()


✅ SUCCESS: /home/hyunseok/BARC/results/success_code/320c0630-98a9-49ed-a53c-b36edb5615b0.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/dbaa0ce1-ce8a-4100-a2a9-dc6763775db5.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/da027b99-0ab1-4790-a777-ca2e22d61aa1.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/16cacd30-8c6f-4a87-8eb6-0d3bbecfd194.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/cc6dd381-2c0b-45e7-9c2f-ada0762aaf82.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/cae0dfc9-7f14-4c8a-8f1f-7b205279bd89.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/28f5af79-42c9-460d-bafa-027cc37dfc79.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/c9657a6a-8323-4bf9-9f78-b6a60b1d985b.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/153b63bb-05ab-457e-9a7f-bdedbb200eca.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_code/fceea498-573b-4ce2-9be1-990604eb0aad.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_cod

In [ ]:
import pygame
import random

# Window size
WIDTH, HEIGHT = 800, 600

class Asteroid:
    def __init__(self):
        self.x = random.randint(0, WIDTH)
        self.y = random.randint(-100, 0)  # Start above screen
        self.vx = random.uniform(-3, 3)
        self.vy = random.uniform(1, 4)
        self.color = (random.randint(50,255), random.randint(50,255), random.randint(50,255))
        self.trail = []
    
    def update(self):
        # Apply gravity
        self.vy += 0.5
        
        self.x += self.vx
        self.y += self.vy
        
        if self.y > HEIGHT - 10:
            self.bounce()
            
        self.add_trail((self.x, self.y))
        
        # Screen wrap for x-axis
        if self.x < 0: self.x = WIDTH
        if self.x > WIDTH: self.x = 0
    
    def bounce(self):
        self.vy = -self.vy * 0.7  # Damping
        self.vx *= random.uniform(0.8, 1.2)  # Random horizontal kick
        
    def add_trail(self, pos):
        self.trail.append((pos[0], pos[1], len(self.trail)))
        if len(self.trail) > 20:
            self.trail.pop(0)
    
    def draw(self, screen):
        for i, (x, y, t) in enumerate(self.trail):
            size = int(10 * (1 - t/30))  # Fade trail
            alpha = int(255 * (1 - t/30))
            
            s = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            s.fill((0,0,0,0))
            for dx in (-size//2, size//2):
                for dy in (-size//2, size//2):
                    if 0 <= x+dx < WIDTH and 0 <= y+dy < HEIGHT:
                        s.set_at((x+dx, y+dy), (self.color[0], self.color[1], self.color[2], alpha))
            screen.blit(s, (0,0))

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

asteroids = [Asteroid() for _ in range(10)]

running = True
while running:
    screen.fill((0, 0, 0))
    
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            asteroids.append(Asteroid())
            
    # Update and draw asteroids
    for asteroid in asteroids:
        asteroid.update()
        asteroid.draw(screen)
    
    # Draw particles from collisions (optional enhancement)
    for _ in range(5):
        x, y = random.randint(0, WIDTH), HEIGHT - 1
        pygame.draw.line(screen, (255, 165, 0), (x,y), (x+random.uniform(-2,2), y-10))
    
    pygame.display.flip()
    clock.tick(60)

pygame.quit()

In [ ]:
import os
import glob
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed

# 1. 파라미터 설정
TEST = "float-line"      # --test 에 들어갈 값
MODEL = "o3-mini"        # --prompt_model 에 들어갈 값
TARGET = "0"             # 출력 디렉터리 명에 포함될 값

# 2. 입력 파일 목록 준비
input_pattern = f"/home/hyunseok/BARC/results/success_problem/*.jsonl"
filenames = glob.glob(input_pattern)
if not filenames:
    print(f"[WARN] 입력 파일을 찾을 수 없습니다: {input_pattern}")
    exit(1)

# 3. 출력 및 실패 로그 디렉터리 생성

output_dir  = f"/home/hyunseok/BARC/results/success_problem"
failure_dir = f"/home/hyunseok/BARC/results/failure_problem"

failure_dir = f"/home/hyunseok/BARC/results/failure_vis"
output_dir=f"/home/hyunseok/BARC/results/success_vis"

os.makedirs(output_dir,  exist_ok=True)
os.makedirs(failure_dir, exist_ok=True)

def log_error(jsonl_name, error_message, gif_id, result_path):
    from hyeonseok_utils.generate_metadata_desc import generate_metadata_csv_of_step_descriptions
    METADATA_CSV_PATH='/home/hyunseok/BARC/results/metadata/step_problem_metadata.csv'
    with open(result_path, "w") as f:
        import uuid
        from datetime import datetime, timezone
        step_id = str(uuid.uuid4())
        try:
            sample_records = [{
                "id": step_id,
                "step_name": "problem",
                "prev_step_id": "GIF",
                "gif_id": str(gif_id),
                "gen_model": "o3-mini",
                "result_code":0,
                "result_path": result_path,
                "error_message": str(error_message),
                "createAt": datetime.now(timezone.utc),   # ← 현재 UTC 시각
                # "token_usage": {"prompt": 142, "completion": 23},
            }]
            generate_metadata_csv_of_step_descriptions( sample_records, output_csv=METADATA_CSV_PATH )
        except Exception as e:
            print(f"an error occurred: {jsonl_name}, {e}")
import os
from hyeonseok_utils.csv_key_unique_check import find_value_in_column
def get_gif_id_metadata_problem(id):
    gif_base_dir = './data/GIF'
    path='/home/hyunseok/BARC/results/metadata/step_problem_metadata.csv' 
    
    # 예시1: user_id 열에서 값 찾기
    result = find_value_in_column(path, column="id", value=id)
    # print(f"[열 검사] 유일 여부: {result}")
    gif_id = result['records'][0]['gif-id']
    gif_path = os.path.join(gif_base_dir, gif_id+'.gif')
    if os.path.exists(gif_path):
        return gif_id+'.gif'
    
def process_file(fn: str):
    fileid = os.path.splitext(os.path.basename(fn))[0]
    gif_id = get_gif_id_metadata_problem(fileid)
    """
    단일 .jsonl 파일에 대해 generate_problems.py를 실행합니다.
    """
    cmd = [
        "python", "visualize_problems.py",
        "--jsonl", fn,
        "--gifid", gif_id,
        "--outdir", output_dir,
        
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return fn, result.returncode, result.stdout, result.stderr

def main():
    folder = "/home/hyunseok/BARC/results/success_problem"
    csv_file = "/home/hyunseok/BARC/data/records.csv"
    csv_file='/home/hyunseok/BARC/results/metadata/step_problem_metadata.csv'
    base_dir='/home/hyunseok/BARC/results/success_problem'

    # 동시 실행 워커 수 (파일 개수나 머신 사양에 맞게 조절)
    max_workers = min(12, len(filenames))
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = { executor.submit(process_file, fn): fn for fn in filenames }
        for future in as_completed(futures):
            fn = futures[future]
            try:
                filename, code, out, err = future.result()
                if code == 0:
                    print(f"✅ SUCCESS: {filename}")
                else:
                    print(f"❌ FAIL:    {filename} (exit {code})")
                    # 실패 로그 파일로 저장
                    base = os.path.splitext(os.path.basename(filename))[0]
                    log_path = os.path.join(failure_dir, f"{base}.log")
                    with open(log_path, "w", encoding="utf-8") as logf:
                        logf.write(f"COMMAND: python visualize_problems.py --jsonl {filename} --outdir {output_dir}  \n")
                        logf.write(f"EXIT CODE: {code}\n\n")
                        logf.write("--- STDOUT ---\n")
                        logf.write(out + "\n")
                        logf.write("--- STDERR ---\n")
                        logf.write(err + "\n")
                    print(f"    👉 로그 저장: {log_path}")
            except Exception as e:
                print(f"❌ EXCEPTION: {fn} -> {e}")

if __name__ == "__main__":
    main()


❌ EXCEPTION: /home/hyunseok/BARC/results/success_problem/0bc3b679-7f84-45a4-a41b-945d27d91214.jsonl -> list index out of range
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/ca484868-c553-4963-99d7-8b78147bc02e.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/7ae27a50-0daf-4cd9-9708-275fa6423ebc.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/b7012c7f-8259-441b-9a89-5045ab491fa1.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/ee93ad5d-ace5-4d5d-b965-9ccb87dd0d68.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/c236bb41-b289-4bca-a6d6-f49cea6eed51.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/54e8c8d9-2f0b-4e55-9d45-d245442df2ca.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/399c938c-457f-4eb3-a09b-bf1c890d3f12.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/926ecf8f-03c8-412a-a913-52aff97f6219.jsonl
✅ SUCCESS: /home/hyunseok/BARC/results/success_problem/6bed56b4-8a46-4ec4-b6f5-3a2d88922a

In [6]:
import os
import glob
import pandas as pd

def find_empty_jsonl(folder_path: str) -> list[str]:
    """
    folder_path 내의 모든 .jsonl 파일 중
    크기가 0인(빈) 파일의 basename 리스트를 반환.
    """
    empty_files = []
    pattern = os.path.join(folder_path, "*.jsonl")
    for filepath in glob.glob(pattern):
        if os.path.getsize(filepath) == 0:
            empty_files.append(os.path.basename(filepath))
    return empty_files


def search_filenames_in_csv(csv_path: str, filenames: list[str]) -> dict[str, list[str]]:
    """
    filenames 리스트에 든 각각의 이름이
    csv_path 파일의 어떤 컬럼에서 발견되는지 찾아서
    { filename: [column1, column2, ...], ... } 형태로 반환.
    """
    df = pd.read_csv(csv_path, dtype=str, keep_default_na=False)
    result = {fname: [] for fname in filenames}

    for col in df.columns:
        # 그 컬럼에 filenames 중 어떤 값이 있는지 체크
        mask = df[col].isin(filenames)
        if mask.any():
            found = set(df.loc[mask, col])
            for fname in found:
                result[fname].append(col)

    return result


def modify_csv_values(
    csv_path: str,
    target_column: str,
    old_values: list[str],
    new_value: str,
    output_path: str | None = None
) -> None:
    """
    csv_path 를 읽어서,
    target_column 에 old_values 중 하나인 모든 셀을 new_value 로 바꾼 뒤
    (덮어쓰기 혹은 output_path로 저장)
    """
    df = pd.read_csv(csv_path, dtype=str, keep_default_na=False)

    # 변경할 마스크 생성
    mask = df[target_column].isin(old_values)
    if mask.any():
        df.loc[mask, target_column] = new_value
        save_path = output_path or csv_path
        df.to_csv(save_path, index=False)
        print(f"[✔] {len(df[mask])}개 행의 `{target_column}` 값을 '{new_value}'로 변경하여 저장: {save_path}")
    else:
        print(f"[–] `{target_column}` 컬럼에 '{old_values}'에 해당하는 값이 없습니다.")


if __name__ == "__main__":
    # 예시 사용법
    folder = "/home/hyunseok/BARC/results/success_problem"
    csv_file = "/home/hyunseok/BARC/data/records.csv"
    csv_file='/home/hyunseok/BARC/results/metadata/step_problem_metadata.csv'

    # 1) 빈 .jsonl 파일 찾기
    empty_list = find_empty_jsonl(folder)
    print("빈 파일 목록:", empty_list)

    # 2) CSV에서 이 파일명이 어떤 컬럼에 들어있는지 찾기
    # occurences = search_filenames_in_csv(csv_file, empty_list)
    # print("파일명 등장 컬럼 매핑:", occurences)

    # # 3) 예) 'error_log' 컬럼에 들어있는 파일명을 'MISSING'으로 통일
    # #    (실제로 바꾸고 싶다면 occurences 중 컬럼명을 골라 사용)
    # if occurences:
    #     modify_csv_values(
    #         csv_path=csv_file,
    #         target_column="error_log",    # ← 수정할 컬럼명
    #         old_values=empty_list,
    #         new_value="MISSING",
    #         output_path=None                # None이면 원본 덮어쓰기
    #     )


빈 파일 목록: ['d2d2d5a1-9a21-49a9-b7bd-c7adeea8868b.jsonl', '02a9c325-9365-407a-9b98-2ca878971787.jsonl', '6093d90e-b930-4f09-bcef-75cfc62600a5.jsonl', 'ae0199b7-77b9-452f-aeab-fee250cfc178.jsonl', '429bcd61-a27c-425a-a02f-707e0d4bbab5.jsonl', 'e04145ff-470e-445d-83f4-ea8eb2379cb8.jsonl', '24198976-8572-4f81-8d4d-d3fcfbbaac40.jsonl', '5415b10b-fae0-429d-b2b6-37cb62c3dc0c.jsonl', '1c72ffac-a230-431a-a2c5-7f7cd53320a0.jsonl', 'ab40f8a0-7a6c-4df7-9a37-0d18a4c86633.jsonl', '54f2fc0a-9dd2-45f6-b0f2-722367f1812b.jsonl', 'd41c66a3-c364-42ca-8610-77d8d6a01c9a.jsonl', '3877a322-fa36-45c0-b00a-b0d5c412c6d7.jsonl', '2149d793-45d2-4a7d-9bed-ca8a4d1f8167.jsonl', '329e0d42-2694-4d80-b6ee-2c5e9304dd81.jsonl', 'b6c52462-7821-4dea-9b40-21e69fa43955.jsonl', '1551b682-3539-45d3-b826-99c8c20b78f7.jsonl', '5a52c83a-8b4d-438b-8819-af039cb8e6e9.jsonl', '86270830-57b7-4e32-91b2-762f6dfa69af.jsonl', 'a0f1e6a0-0369-4cb2-a2f9-271bfeb864dc.jsonl', '7d9001c1-0f96-432b-99ad-983f62da48ed.jsonl', '7de9f4d9-a07f-466f-aa6f

In [ ]:
# 예시 사용법
folder = "/home/hyunseok/BARC/results/success_problem"
csv_file='/home/hyunseok/BARC/results/metadata/step_problem_metadata.csv'
base_dir='/home/hyunseok/BARC/results/success_problem'
# 1) 빈 .jsonl 파일 찾기
empty_list = find_empty_jsonl(folder)
empty_file_path = [ os.path.join(base_dir, ffpath) for ffpath in empty_list]
print("빈 파일 목록:", len(empty_file_path))


빈 파일 목록: 43


In [ ]:
import os
from hyeonseok_utils.csv_key_unique_check import find_value_in_column
if __name__ == "__main__":
    base_dir = '/home/hyunseok/BARC/results/success_problem'
    path='/home/hyunseok/BARC/results/metadata/step_problem_metadata.csv'
    val  = '0a0ca1f6-1545-4c57-b989-48f8489fec0e'

    # 예시1: user_id 열에서 값 찾기
    result = find_value_in_column(path, column="id", value=val)
    print(f"[열 검사] 유일 여부: {result}")
    prob_path = os.path.join(base_dir, result['records'][0]['id']+'.jsonl')
    print(prob_path)
    if os.path.exists(prob_path):
        print(prob_path)
 

[열 검사] 유일 여부: {'unique': False, 'records': []}


IndexError: list index out of range

In [2]:
!pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 14.1 MB/s eta 0:00:00a 0:00:01


pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


TypeError: 'float' object cannot be interpreted as an integer

In [4]:
pygame.quit()